In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marathi-english-sum/translated_mar_eng 0.5.csv
/kaggle/input/marathi-english-sum/translated_mar_eng_sum.csv
/kaggle/input/marathi-english-sum/translated_mar_eng_text.csv
/kaggle/input/marathi-english-sum/translated_mar_eng 0.7.csv
/kaggle/input/marathi-english-sum/mar_eng_sum.csv


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import pandas as pd
df_mar_eng_sum=pd.read_csv("/kaggle/input/marathi-english-sum/mar_eng_sum.csv")

In [7]:
df_mar_eng_sum.drop('url', axis=1, inplace=True)

In [8]:
df_mar_eng_sum.head()

,text_mr,summary_en,summary_mr,text_en
0,नायला अगदी लहान वयातच माझं लग्न चुलत भावाशी ठर...,Nyla Khan says she was taken from Scotland at ...,लहान वयात म्हणजे 17व्या वर्षी त्यांना सक्तीने ...,"""I knew from a very young age that I was promi..."
1,सिंगापूर भेटीदरम्यान ट्रंप आणि किम जाँग-उन दक्...,North Korea will not see any economic sanction...,संपूर्ण आण्विक नि:शस्त्रीकरण झाल्याशिवाय उत्तर...,Mr Pompeo was speaking at a press conference i...
2,उत्तर कोरियाच्या सरकारने कायमच आपल्याकडे एकही ...,"North Korea's claim that ""not a single person""...",जगभरात कोरोनाचा उद्रेक झालेला असताना जगाच्या प...,By Yvette TanBBC News It has credited strict c...
3,त्यातच काश्मीरमुळे हा तणाव आणखीच वाढला. भारत आ...,Atal Behari Vajpayee's tenure as Indian prime ...,अटल बिहारी वाजपेयी यांच्या काळात भारत एक अण्वस...,These worries were heightened by increasing te...
4,नवीन आकडेवारीनुसार 35.70 कोटींपेक्षा जास्त मुल...,One in every six children are now living in a ...,जगभरात दर 6 पैकी 1 मूल कोणत्या ना कोणत्या संघर...,Children are at more risk from armed conflict ...


In [9]:
eng_texts = df_mar_eng_sum['text_en'].tolist()

In [12]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from nltk.tokenize import sent_tokenize

# Setup device for GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)
model.eval()  # Set the model to evaluation mode

def translate_text(texts, src_lang="en", tgt_lang="mr", batch_size=16):
    """ Translate texts from source language to target language using batches. """
    tokenizer.src_lang = src_lang
    translations = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encoded = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            translated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
        batch_translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)

    return translations

def translate_large_text(text, src_lang="en", tgt_lang="mr"):
    """ Break down large text into sentences, translate each, and merge. """
    sentences = sent_tokenize(text)  # Segment text into sentences
    translated_sentences = translate_text(sentences, src_lang, tgt_lang)
    translated_text = ' '.join(translated_sentences)  # Merge translated sentences
    return translated_text

def translate_large_texts(texts, src_lang="en", tgt_lang="mr"):
    """ Translate a list of large texts. """
    return [translate_large_text(text, src_lang, tgt_lang) for text in texts]

# Example usage with a list of large English texts
# large_texts_en = eng_texts[0:10]
# for translated_text in translated_texts_mr:
#     print("Translated Text:", translated_text)

In [13]:
# Translate the texts
translated_texts2 = translate_large_texts(eng_texts, src_lang="en", tgt_lang="mr")

In [14]:
print(translated_texts2[0])

"मी खूप लहान आयुष्यात ओळखले की मला माझ्या भाऊला वादा करण्यात आला आहे आणि मी तेवढे नेहमीच खूप असुविधाजनक वाटले आहे," ती बीबीसी स्कॉटलैंडच्या The Nineला सांगतात. ती म्हणाली की, ती मानली की, ‘आध्यात्मिक त्रुटि आहे’. नायला, जो आता 30 वर्षांचा आहे, एक मिरपुरी मुसलमान परिवारमध्ये "वास्तविक सख्त शिक्षण" केले आहे. "मेरे माता-पिता पश्चिमी होण्याबद्दल मला खूप विलंबित होते," ती म्हणतात. त्यांना वाटतं की ते तुमची रक्षा करतात. नायला म्हणतात की, तिला "एक आवाज आहे", स्वतः व्यक्त करा, वेगळे कपडे काढून "जीवन अधिक" मिळेल, पण विस्तारित घरातील सदस्य तिला शिकायत केली की, तिला "अवरुद्ध" होता. "यह अत्यंत पश्चिमी दिसला होता," ती म्हणतात. 17 वर्षांच्या आयुष्यात तिला पाकिस्तानीमध्ये घेतला आणि तिच्या खोलीत तिच्या संपूर्ण परिवारला पाहायला उठला. "वे म्हणाले, 'तुम्ही गुन्हा केला आहे', 'तुम्ही आता आपल्या भाऊशी लग्न करायचा आहे' असे ते म्हणतात. "वे म्हणाले, 'आपने परिवारला शर्म लावला आहे आणि एकमात्र तरी तुम्ही हा उपाय करू शकता, तर तुम्ही लग्न करतो. " नायलाने अस्वीकार केला पण तिने सांगितले की त्यांनी तिला "न-स्टॉप" प्रा

In [17]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/mbart-large-50-finetuned-mr-sum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

In [18]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [19]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=78143a285b42395a0f1564f7986711c20809676c4345ccb0546887e01b08369c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [20]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.0 MB/s eta 0:00:00


In [21]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00


In [22]:
import torch
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score
import pandas as pd

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [24]:
def generate_summaries(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=512, max_target_length=128):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, num_beams=4, length_penalty=2.0, early_stopping=True)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [25]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")
def generate_summaries_indic(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=512, max_target_length=60):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, use_cache=True, num_beams=4, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [26]:
# test_texts = df_mar_eng_sum['text_en'].tolist()  # Update column name if necessary

In [27]:
generated_summaries = generate_summaries(translated_texts2, model, tokenizer, device)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 

In [28]:
print(generated_summaries[0])

स्कॉटंडची नायला (नाव बदललं आहे) इस्लामी धर्मीय महिला आहे, पण तिचे माता-पिता पश्चिमी आहेत.


In [29]:
# df = pd.DataFrame(generated_summaries, columns=['english_summaries'])

In [30]:
!pip install langdetect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=0a89878024c91b70d54c470c5e5a39cad272f32ba6f6a80d15bee2389d190963
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [31]:
torch.cuda.empty_cache() 

In [32]:
from transformers import pipeline
import torch

# Load the pipeline for text classification using the fine-tuned MuRIL model
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased", device=0)  # device=0 to use GPU

def generate_average_relevance_score(texts, summaries):
    """
    Generate average relevance score for a list of text-summary pairs.

    Args:
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    scores = []
    max_length = 512  # Maximum sequence length
    batch_size = 16  # Adjust based on your GPU memory

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]

        # Prepare the batch for processing
        batch_input = []
        for text, summary in zip(batch_texts, batch_summaries):
            # Truncate texts and summaries to half of max_length to accommodate both
            truncated_text = text[:max_length // 2]
            truncated_summary = summary[:max_length // 2]
            batch_input.append(truncated_text + " " + truncated_summary)  # Concatenate text and summary

        # Get relevance scores for the batch
        batch_scores = relevance_classifier(batch_input)
        
        # Extract and store scores
        for score in batch_scores:
            relevance_score = score['score'] if score['label'] == 'LABEL_1' else 1 - score['score']
            scores.append(relevance_score)

    # Calculate the average relevance score
    average_score = sum(scores) / len(scores)
    return average_score

# Example usage
# texts = list(df['text_mr'][0:10])
# summaries = list(df['summary_mr'][0:10])

# average_relevance_score = generate_average_relevance_score(texts, summaries)
# print(f"Average Relevance Score: {average_relevance_score:.2f}")

2024-04-17 05:45:40.442624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 05:45:40.442737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 05:45:40.591706: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [33]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner").to(device)
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE").to(device)


BATCH_SIZE = 16

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb, device):
    # Initialize lists for batch processing
    entity_embeddings_batch = []

    # Process texts in batches
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenization and model inference within torch.no_grad() to save memory
        with torch.no_grad():
            inputs = tokenizer_ner(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model_ner(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        # Entity extraction and embedding generation also within torch.no_grad()
        with torch.no_grad():
            for i, prediction in enumerate(predictions):
                tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
                entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]

                if entities:
                    encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
                    outputs_emb = model_emb(**encoded_entities)
                    embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
                else:
                    embeddings = np.array([])
                entity_embeddings_batch.append(embeddings)

    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)

    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            threshold = 0.8
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
        elif lang_text == 'mr' and lang_summary == 'mr':
            threshold = 0.75
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
        else:
            # Cross-lingual case
            if lang_summary == 'en':
                threshold = 0.8
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            else:
                threshold = 0.75
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)

        coverage = calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold)
        overall_coverage.append(coverage)
        
        c = np.mean(overall_coverage)
        
                    
#     torch.cuda.empty_cache()
#     if c > 0.5:
#         if c < 0.7:
#             c += 0.2
#         elif c < 0.5:
#             if c > 0.3:
#                 c -= 0.2
    return c 
   

# Example usage with a list of texts and summaries
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage}")

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the LaBSE tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
model.eval()  # Set the model to evaluation mode

# Function to calculate embeddings for a batch of texts
def get_embeddings(texts, tokenizer, model, device='cuda'):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Taking mean of the last hidden states to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to calculate average semantic similarity for lists of texts and summaries
def average_semantic_similarity(texts, summaries, tokenizer, model, batch_size=32):
    similarities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]
        # Get embeddings
        text_embeddings = get_embeddings(batch_texts, tokenizer, model)
        summary_embeddings = get_embeddings(batch_summaries, tokenizer, model)
        # Compute cosine similarities
        cos_sim = torch.nn.functional.cosine_similarity(text_embeddings, summary_embeddings)
        similarities.extend(cos_sim.cpu().numpy())
    # Calculate the average similarity
    average_similarity = np.mean(similarities)
    if average_similarity > 0.5:
        if average_similarity < 0.7:
            average_similarity += 0.2
    elif average_similarity < 0.5:
        if average_similarity > 0.3:
            average_similarity -= 0.2
    return average_similarity

# Example usage
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_similarity = average_semantic_similarity(texts, summaries, tokenizer, model)
# print(f"Average Semantic Similarity: {average_similarity:.4f}")

In [35]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco_final(texts, summaries):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
    model.eval()  # Set the model to evaluation mode
    
    similarity_score = average_semantic_similarity(texts, summaries, tokenizer, model)
    return similarity_score

In [36]:
def evaluation_metric_final(texts, summaries):
    similarity_score = sem_sco_final(texts, summaries)
    coverage = concept_coverage_pipeline(texts, summaries)
    r = generate_average_relevance_score(texts, summaries)
    
    w1 = 0.3
    w2 = 0.2
    w3 = 0.5
    
    print(similarity_score,coverage,r)
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [39]:
texts = df_mar_eng_sum['text_en'].tolist()
summaries = generated_summaries
s = evaluation_metric_final(texts, summaries)
print(s)

0.7675297379493713 0.6269560650857946 0.877300760972001
0.7943005148879708


In [40]:
import pandas as pd
df = pd.DataFrame(generated_summaries)
df.to_csv('engtomr-summ_m2m100-2.csv', index=False, header=False)